# Engine Prognosis Modelling Streamlit frontend

Some of the meain features from FD001 will be represented by using streamlit tools, looking for more visual and interactive environmet to analyse the different engine's features.

Last, the user will be able to use one of the best models designed in FD001 to perform direct predictions in engines which have not yet failed, and by selecting the cycle number, it will provide the expected remaining useful life.
Additionally, this environment provides the possibility to include a sudden issue noticed in last flight monitored, which cases an increase in several sensor measurements due to the malfunction of related engine components. This specific values increase will provide new values for the predicted RUL, and the tool will notice the user if the issue found hastens the need of a precocious maintenance

In [30]:
# First step is to install necessary package to visualize plots in streamlit
!pip install plotly

In [134]:
%%writefile app.py

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
import altair as alt

st.title("Engine Prognosis Modelling Frontend")
st.write("It has been developed a simple interactive frontend to allow the user to play a bit \
        with some variables from the dataset")

# Preparation for features visualization
df = pd.read_pickle("df_stream.pkl")
df1  = df[df["ID"]==1]

st.table(df1.head())
st.write("Above, a sample is exposed with all the monitored variables from 100 engine units along their \
        operational lifes")

df_max_cycles = df.groupby("ID").max("Cycle").reset_index()["Cycle"]
st.bar_chart(df_max_cycles)
st.write("These engines have been performing until their end of life, as each engine is different from the others \
        each unit will fail during performance at a different flight cycle. \
        The chart above shows the maximum number of cycles reached by each engine unit")

st.subheader("Operational settings and sensor variation per engine")
st.write("Along their operational lifes, the sensors located throughout the different engine stages \
have managed to catch different magnitudes along the performance which, in these cases, where measurements \
have been already filtered for this study, it is to be noticed the increasing or decreasing trend of the sensors \
which shows the incoming cycle of failure from the engine")

st.write("The user now can have a vision of each of these sensors evolution, by selecting both an engine \
unit and the feature to visualize")

engine_option = df["ID"].unique().tolist()
ids_ = st.selectbox("Which engine unit to display?", engine_option,0)
df_id=df[df["ID"]==ids_]

filtered = st.multiselect("Choose features to display", options=list(df.columns), default=list(df.columns))
df_id_fil = df_id[filtered]

fig1 = px.line(df_id_fil)
st.write(fig1)


st.title("Remaining useful life prediction")
st.subheader("Specific remaining useful life prediction when engine, cycles and sensors damage are fixed")

st.write("Imagine you are a maintenance engineer working in the hangar of an airline. \
       It is into your work schedule to receive aircrafts after each flight cycle, to perform \
       a basic inspection of sensor measurements, \
       as you have developed your prediction model based on historical data, you are now able to apply it \
       for this preliminary check of sensor magnitudes, and determine whether the engine is able to continue performing \
       or perhaps it should need a deeper maintenance check now or in the inmediate future")

st.text("Choose engine number")
engine_option_ = df["ID"].unique().tolist()
id_ = st.selectbox("Which engine unit to display?", engine_option_,1)
df_id_=df[df["ID"]==id_]

cycle_option = df_id_["Cycle"].unique().tolist()
cycles = st.selectbox("How many flights the engine has performed?", cycle_option,1)


X_train = df.drop(columns=["ID","RUL"])
y_train = df["RUL"]
df_test = df[df["ID"]==id_].drop(columns=["ID","RUL"])
X_test = df_test[df_test["Cycle"]==cycles]
sensors = ["sensor_measurement_11","sensor_measurement_3","sensor_measurement_4"]



df_affected = pd.read_pickle("df_stream.pkl")
percents = np.arange(1,1.020,0.005).tolist()

n_estimators = 64
max_depth = 9
min_samples_leaf = 10

hyper_regr = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, min_samples_leaf=min_samples_leaf)
hyper_regr.fit(X_train, y_train)

preds = hyper_regr.predict(X_test).round(0)[0]

st.write("Based on historical experience, this engine shall continue performing without any expected issue \
the following number of flights: ", preds)

if preds <=10:
    st.write("This engine needs maintenance now!")
elif 0 < preds < 50:
    st.write("maintenance shall be schechuled soon")
else:
    st.write("engine should perform good so far")
# Introduce the sudden damage for specific sensor measurements

st.subheader("Addition of sudden sensor components")
st.write("In this theorical situation, imagine that, during the last flight, the pilot has reported a \
noticed malfunction in a specific stage of the engine, which has been alerted due to an increase in 3 \
specific sensor measurements: 3, 4 and 11 (located closely to Low Pressure Turbine)")
st.write("Your mission now is, by using the trained model, determine if these sensor values augmentation \
would carry an extra risk for flight safety from now on, and, depending on the number flights already performed, \
conclude if the engine remaining cycles number forecasted remains into security thresholds, or the initially expected \
maintenance shall be overtaken")

affect = st.selectbox("Sensors measurement value percentage augmented %", percents,1)

for sensor in sensors:
    df_affected[sensor] = df_affected[sensor]*affect

df_test_affected = df_affected[df_affected["ID"]==id_].drop(columns=["ID","RUL"])
X_test_affected = df_test_affected[df_test_affected["Cycle"]==cycles]

sensor_check = st.selectbox("Which sensor to display value increase due to reported failure?", sensors,1)


preds_affected = hyper_regr.predict(X_test_affected).round(0)[0]




if preds_affected <=10:
    st.write("This engine needs maintenance now!")
elif 0 < preds_affected < 50:
    st.write("With these issues, this engine shall continue performing without any further serious risk \
the following number of flights: ", preds_affected,
            "however, a deeper maintenance shall be schechuled soon")
else:
    st.write("With these issues, this engine shall continue performing without any further serious risk \
    the following number of flights", preds_affected,
            "Hence, engine should perform good so far")

fig100 = px.line(        
        df_test, #Data Frame
        x = "Cycle", #Columns from the data frame
        y = sensor_check,
        title = "Sensor Measurement")

fig100.add_scatter(x=df_test_affected['Cycle'], y=df_test_affected[sensor_check], name="sensor issued")

st.plotly_chart(fig100)


Overwriting app.py


In [135]:
!streamlit run app.py

^C
